In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from hazm import Normalizer, word_tokenize
import faiss
import torch
import pickle



C:\Users\software2\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3550: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:


model_id = "vahidhoseini/qwen-roshdv1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9,
)

llm = HuggingFacePipeline(pipeline=pipe)

C:\Users\software2\AppData\Local\Temp\2\ipykernel_3168\1197029383.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model='nvidia/llama-3.1-nemotron-70b-instruct' , temperature=0.3, api_key=api_key, base_url="https://openrouter.ai/api/v1")


In [ ]:
loader = PyPDFLoader("C:/Users/software2/Desktop/chatbot/file.pdf")
docs = loader.load()
num_pages = len(docs)
print(f"The PDF contains {num_pages} pages.")

The PDF contains 101 pages.


#more perftional

#more perfetional

In [ ]:


if isinstance(docs[0], str):
    docs = [Document(page_content=text) for text in docs]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", ".", "؛", ";", " ", ",", "-"],
    keep_separator=True,
    length_function=lambda x: len(x.split())
)

normalizer = Normalizer()

def preprocess(text):
    """پیش‌پردازش متن: نرمال‌سازی و توکن‌سازی."""
    try:
        text = text.lower()
        text = normalizer.normalize(text)
        text = ''.join(e for e in text if e.isalnum() or e.isspace())
        tokens = word_tokenize(text)
        return ' '.join(tokens)
    except Exception as e:
        print(f"Error in preprocessing: {str(e)}")
        return ""

splits = text_splitter.split_documents(docs)
texts = [preprocess(doc.page_content) for doc in splits]

model_name = "Msobhi/Persian_Sentence_Embedding_v3"
embedding_model = HuggingFaceEmbeddings(model_name=model_name, encode_kwargs={
    "batch_size": 16,
    "normalize_embeddings": True,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "padding": True,
    "truncation": True,
    "max_length": 512
})

embeddings_result = embedding_model.embed_documents(texts)


with open('embeddings_result.pkl', 'wb') as file:
    pickle.dump(embeddings_result, file)


C:\Users\software2\AppData\Local\Temp\2\ipykernel_3168\3550581144.py:30: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=model_name, encode_kwargs={
c:\Users\software2\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
FAISS_SETTINGS = {
    "metric": "L2",  
    "index_type": "Flat",  
    "k": 10  
}

d = len(embeddings_result[0])

if FAISS_SETTINGS["metric"] == "L2":
    metric = faiss.METRIC_L2
elif FAISS_SETTINGS["metric"] == "IP":
    metric = faiss.METRIC_INNER_PRODUCT
elif FAISS_SETTINGS["metric"] == "Cosine":
    metric = faiss.METRIC_INNER_PRODUCT
    embeddings_result = [vec / (sum(v ** 2 for v in vec) ** 0.5) for vec in embeddings_result]

if FAISS_SETTINGS["index_type"] == "Flat":
    index = faiss.IndexFlat(d, metric)
else:
    raise ValueError(f"Only 'Flat' index type is supported in this version.")

docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(splits)})

index_to_docstore_id = {i: str(i) for i in range(len(splits))}

vectorstore = FAISS(
    embedding_model.embed_query,
    index,
    docstore,
    index_to_docstore_id
)

vectorstore.add_texts(texts, metadatas=[{"source": f"doc_{i}"} for i in range(len(texts))])

faiss.write_index(vectorstore.index, "faiss_index.index")

with open("faiss_data.pkl", "wb") as f:
    pickle.dump({
        "docstore": vectorstore.docstore,
        "index_to_docstore_id": vectorstore.index_to_docstore_id
    }, f)



`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


### embding_results

In [ ]:
with open('embeddings_result.pkl', 'rb') as file:
    embeddings_resultt = pickle.load(file)


### vectordatabase

In [ ]:
faiss_index = faiss.read_index("faiss_index.index")

with open("faiss_data.pkl", "rb") as f:
    data = pickle.load(f)

from langchain.vectorstores import FAISS
vectorstore = FAISS(
    embedding_function=embedding_model.embed_query,  
    index=faiss_index,
    docstore=data["docstore"],
    index_to_docstore_id=data["index_to_docstore_id"]
)


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [17]:
retriever = vectorstore.as_retriever(
    search_type="similarity",  
    search_kwargs={"k": 10}    
)

In [18]:
retriever

VectorStoreRetriever(tags=['FAISS'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028BA65ABD90>, search_kwargs={'k': 10})

In [16]:
system_prompt = (
    "تو یک دستیار خوب برای سوال و جواب هستی. "
    "از context و سوال و جواب‌های موجود برای پاسخ دادن استفاده کن. "
    "لطفاً تنها اطلاعات مرتبط با سوال را از context استخراج کن و از اطلاعات بی‌ربط خودداری کن. "
    "اگر جواب سوالی را نمی‌دانی، به جای اینکه بگویی 'نمی دانم'، بگو که 'پاسخ دقیقی برای این سوال ندارم' یا درخواست اطلاعات بیشتر کن."
    "به زبان فارسی جواب بده."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [12]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)


# output 

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

query = ' برنامه نویسی پایتون بلدی؟'
result = rag_chain.invoke({"input": query})

answer = result["answer"]
print("Answer:", answer)